<a href="https://colab.research.google.com/github/Grace-Gitome/DS_Track/blob/main/STAGE_B_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning: Regression - Predicting Energy Efficiency of Buildings.

### Dataset 
The dataset is the Appliances Energy Prediction data. The data set is at 10 min for about 4.5 months. The house temperature and humidity conditions were monitored with a ZigBee wireless sensor network. Each wireless node transmitted the temperature and humidity conditions around 3.3 min. Then, the wireless data was averaged for 10 minutes periods. The energy data was logged every 10 minutes with m-bus energy meters. Weather from the nearest airport weather station (Chievres Airport, Belgium) was downloaded from a public data set from Reliable Prognosis (rp5.ru), and merged together with the experimental data sets using the date and time column. Two random variables have been included in the data set for testing the regression models and to filter out non predictive attributes (parameters). The attribute information can be seen below.


In [1]:
from google.colab import files
uploaded = files.upload()

Saving energydata_complete.csv to energydata_complete.csv


In [7]:
df = pd.read_csv("energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


### Dataset description

In [8]:
df.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
count,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,...,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000,19735.000000
mean,97.694958,3.801875,21.686571,40.259739,20.341219,40.420420,22.267611,39.242500,20.855335,39.026904,...,19.485828,41.552401,7.411665,755.522602,79.750418,4.039752,38.330834,3.760707,24.988033,24.988033
std,102.524891,7.935988,1.606066,3.979299,2.192974,4.069813,2.006111,3.254576,2.042884,4.341321,...,2.014712,4.151497,5.317409,7.399441,14.901088,2.451221,11.794719,4.194648,14.496634,14.496634
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.463333,17.200000,28.766667,15.100000,27.660000,...,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.000000,-6.600000,0.005322,0.005322
25%,50.000000,0.000000,20.760000,37.333333,18.790000,37.900000,20.790000,36.900000,19.530000,35.530000,...,18.000000,38.500000,3.666667,750.933333,70.333333,2.000000,29.000000,0.900000,12.497889,12.497889
50%,60.000000,0.000000,21.600000,39.656667,20.000000,40.500000,22.100000,38.530000,20.666667,38.400000,...,19.390000,40.900000,6.916667,756.100000,83.666667,3.666667,40.000000,3.433333,24.897653,24.897653
75%,100.000000,0.000000,22.600000,43.066667,21.500000,43.260000,23.290000,41.760000,22.100000,42.156667,...,20.600000,44.338095,10.408333,760.933333,91.666667,5.500000,40.000000,6.566667,37.583769,37.583769
max,1080.000000,70.000000,26.260000,63.360000,29.856667,56.026667,29.236000,50.163333,26.200000,51.090000,...,24.500000,53.326667,26.100000,772.300000,100.000000,14.000000,66.000000,15.500000,49.996530,49.996530


In [10]:
df.info()

# RH - room humidity, T -temperature, rv - random variable

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

### Libraries

In [12]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use("ggplot")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import (LinearRegression,
                                  Ridge,
                                  Lasso)

from sklearn.metrics import (r2_score,
                             mean_absolute_error,
                             mean_squared_error)

# Q17
Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

What is the Mean Absolute Error (in two decimal places)?

In [19]:
#normalizing the dataset
scaler = MinMaxScaler()
normalized_df =pd.DataFrame (scaler.fit_transform(df.drop(columns = ['date','lights'])),
                             columns = df.drop(columns =['date', 'lights'] ).columns)



In [25]:
# segmenting the features and target variabls.
features_df = normalized_df.drop(columns = ['Appliances'])
target_df   = normalized_df.Appliances


In [26]:
#Now we split our dataset into training and testing dataset.
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(features_df, target_df, test_size = 0.3, random_state=42)

### Q16
From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [31]:
linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(x_train[['T2']],x_train.T6)

#obtain the predictions
T6_predicted_values =linear_model.predict(x_test[['T2']])


In [29]:
# Finding r_squared
from sklearn.metrics import r2_score
r2_score = r2_score(x_test.T6, T6_predicted_values)
round(r2_score,2)

0.64

In [34]:
#Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(x_test.T6,T6_predicted_values)
round(mae,2)

0.08

#Q18
What is the Residual Sum of Squares (in two decimal places)?

Options
47.19

45.35

39.20

44.11

In [37]:
#Residual Sum of Squares
RSS = np.sum (np.square(x_test.T6 - T6_predicted_values))
round(RSS,2)

66.12

In [42]:
#Root Mean Squared Error
RMSE = np.sqrt(mean_squared_error(x_test.T6,T6_predicted_values))
round (RMSE,3)

0.106

In [58]:
#Root Mean Squared Error
RMSE = np.sqrt(mean_squared_error(y_test,T6_predicted_values))
round (RMSE,3)

0.362

#Qn 20
What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)

Options
0.094

0.101

0.112

0.089

In [43]:
#L2 regularisation
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 0.4)
print(ridge_reg.fit(x_train, y_train))

Ridge(alpha=0.4)


In [46]:
#Lasso regression /L2 regularisation
from sklearn.linear_model import Lasso
lasso_reg = Lasso (alpha = 0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [48]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

LinearRegression()

In [50]:
#Comparing the effects of regularisation

#this function returns the weight of every feature
def get_weights_df(model, feat, col_name):
  weights = pd.Series(linear_model.coef_, feat.columns).sort_values()
  weights_df =pd.DataFrame(weights).reset_index()
  weights_df.columns =['Features',col_name]
  weights_df[col_name].round(3)
  return weights_df

In [51]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, x_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')
final_weights

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
0,RH_2,-0.456698,-0.456698,-0.456698
1,T_out,-0.321860,-0.321860,-0.321860
2,T2,-0.236178,-0.236178,-0.236178
3,T9,-0.189941,-0.189941,-0.189941
4,RH_8,-0.157595,-0.157595,-0.157595
5,RH_out,-0.077671,-0.077671,-0.077671
6,RH_7,-0.044614,-0.044614,-0.044614
7,RH_9,-0.039800,-0.039800,-0.039800
8,T5,-0.015657,-0.015657,-0.015657
9,T1,-0.003281,-0.003281,-0.003281


#Qn
Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?


T_out, T3

T6, RH_4

rv1, Windspeed

RH_2, RH_1

In [52]:
final_weights.sort_values("Linear_Model_Weight", ascending = False)

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
25,RH_1,0.553547,0.553547,0.553547
24,T3,0.290627,0.290627,0.290627
23,T6,0.236425,0.236425,0.236425
22,Tdewpoint,0.117758,0.117758,0.117758
21,T8,0.101995,0.101995,0.101995
20,RH_3,0.096048,0.096048,0.096048
19,RH_6,0.038049,0.038049,0.038049
18,Windspeed,0.029183,0.029183,0.029183
17,T4,0.028981,0.028981,0.028981
16,RH_4,0.026386,0.026386,0.026386


#Qn

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

Options
Yes

No

In [55]:
y_pred_linear = linear_model.predict(x_test)
y_pred_ridge = ridge_reg.predict(x_test)
y_pred_lasso = lasso_reg.predict(x_test)

In [62]:
RMSE = (np.sqrt(mean_squared_error(y_test,y_pred_linear)))
round(RMSE,3)

0.088

In [57]:
RMSE = (np.sqrt(mean_squared_error(y_test,y_pred_ridge)))
round(RMSE,3)

0.088

In [61]:
RMSE = (np.sqrt(mean_squared_error(y_test,y_pred_lasso)))
round(RMSE,3)

0.094

#Qn

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

Options
4

5

7

26